# 主題 04-1. 用RNN做情意分析

我們終於要介紹三大神經網路的最後一個, 也就是 RNN。RNN 有不少的變型, 例如 LSTM 和 GRU 等等, 不過我們都通稱叫 RNN。RNN 是一種「有記憶」的神經網路, 非常適合時間序列啦, 或是不定長度的輸入資料。

我們來看看怎麼樣用 RNN 做電影評論的「情意分析」, 也就是知道一則評論究竟是「正評」還是「負評」。

## 1. 初始準備

基本上和之前是一樣的, 我們就不再說明。

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shared\ drives/鮭魚卵軍艦/aigo_農業局資料_20190707/source

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/Shared drives/鮭魚卵軍艦/aigo_農業局資料_20190707/source


In [0]:
! wget https://raw.githubusercontent.com/bojone/keras_lookahead/master/lookahead.py

--2019-07-29 06:19:35--  https://raw.githubusercontent.com/bojone/keras_lookahead/master/lookahead.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2582 (2.5K) [text/plain]
Saving to: ‘lookahead.py’

lookahead.py        100%[===================>]   2.52K  --.-KB/s    in 0.001s  

2019-07-29 06:19:35 (3.30 MB/s) - ‘lookahead.py’ saved [2582/2582]



In [0]:
%env KERAS_BACKEND=tensorflow
from lookahead import *

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## 2. 讀入 IMDB 電影數據庫

今天我們要評入 IMDB 電影數據庫影評的部份。

In [0]:
from keras.datasets import imdb

In [0]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 0s 0us/step


ValueError: ignored

要注意這裡我們限制只選「最常用」1 萬字, 也就是超過這範圍的就當不存在。這是文字分析常會做的事。

In [0]:
print("訓練資料總筆數 =", len(x_train))
print("測試資料總筆數 =", len(x_test))

訓練資料總筆數 = 25000
測試資料總筆數 = 25000


### 2.1 輸入資料部份

我們來看一下輸入部份長什麼樣子?

In [0]:
x_train[99]

[1,
 1230,
 3765,
 566,
 97,
 189,
 102,
 86,
 7,
 32,
 4,
 973,
 16,
 55,
 355,
 18,
 14,
 20,
 4,
 64,
 542,
 173,
 16,
 4,
 893,
 2115,
 5376,
 250,
 39,
 8013,
 4,
 1362,
 2,
 14,
 102,
 47,
 57,
 599,
 633,
 6,
 1317,
 2,
 8,
 6,
 189,
 20,
 57,
 206,
 57,
 116,
 5,
 57,
 836,
 82,
 6,
 1317,
 2,
 3728,
 2,
 9,
 6,
 52,
 284,
 21,
 29,
 9,
 38,
 2245,
 5,
 1044,
 11,
 14,
 15,
 45,
 619,
 50,
 71,
 6,
 171,
 531,
 15,
 71,
 424,
 8,
 30,
 163,
 6211,
 4,
 1629,
 189,
 212,
 102,
 5,
 57,
 31,
 1498,
 11,
 4,
 311,
 13,
 197,
 15,
 14,
 20,
 16,
 1150,
 1479,
 5,
 13,
 161,
 990,
 692,
 5,
 1706,
 12,
 69,
 77,
 1194,
 8,
 3245,
 2001,
 553,
 67,
 14,
 20,
 48,
 25,
 423,
 13,
 131,
 124,
 51,
 25,
 122,
 236,
 1506,
 198,
 4,
 64,
 552,
 7,
 415,
 37,
 62,
 169,
 14,
 20,
 60,
 2602,
 629,
 5,
 615,
 14,
 9,
 8,
 25,
 1230,
 3765,
 570,
 231,
 189,
 102,
 14,
 20,
 166,
 2039,
 168,
 40,
 2450,
 5486,
 3298]

注意這其實是一個 list 而不是 array, 原因是每筆資料 (每段影評) 長度自然是不一樣的! 我們檢查一下前 10 筆的長度就可以知道。

In [0]:
type(x_train[99])

list

In [0]:
for i in range(10):
    print(len(x_train[i]), end=', ')

218, 189, 141, 550, 147, 43, 123, 562, 233, 130, 

最後要說明的是, 在每筆輸入資料的數字都代表英文的一個單字。編號方式是在我們資料庫裡所有文字的排序: 也就是出現頻率越高, 代表的數字就越小。

### 2.2 輸出資料部份

輸出方面應該很容易想像, 我們來看看前 10 筆。結果自然就是 0 (負評) 或 1 (正評)。

In [0]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

### 2.3 送入神經網路的輸入處理

雖然 RNN 是可以處理不同長度的輸入, 在寫程式時我們還是要

* 設輸入文字長度的上限
* 把每段文字都弄成一樣長, 太短的後面補上 0

In [0]:
from keras.preprocessing import sequence

In [0]:
x_train = sequence.pad_sequences(x_train, maxlen=100)
x_test = sequence.pad_sequences(x_test, maxlen=100)

In [0]:
x_train.shape

(25000, 100)

In [0]:
x_train[99]

array([  14,   15,   45,  619,   50,   71,    6,  171,  531,   15,   71,
        424,    8,   30,  163, 6211,    4, 1629,  189,  212,  102,    5,
         57,   31, 1498,   11,    4,  311,   13,  197,   15,   14,   20,
         16, 1150, 1479,    5,   13,  161,  990,  692,    5, 1706,   12,
         69,   77, 1194,    8, 3245, 2001,  553,   67,   14,   20,   48,
         25,  423,   13,  131,  124,   51,   25,  122,  236, 1506,  198,
          4,   64,  552,    7,  415,   37,   62,  169,   14,   20,   60,
       2602,  629,    5,  615,   14,    9,    8,   25, 1230, 3765,  570,
        231,  189,  102,   14,   20,  166, 2039,  168,   40, 2450, 5486,
       3298], dtype=int32)

至此我們可以來寫我們的第一個 RNN 了!

## 3. 打造你的 RNN

這裡我們選用 LSTM, 基本上用哪種 RNN 寫法都是差不多的!

### 3.1 決定神經網路架構

* 先將 10000 維的文字壓到 128 維
* 然後用 128 個 LSTM
* 最後一個 output, 直接用 sigmoid 送出

### 3.2 建構我們的神經網路

文字我們用 1-hot 表示是很標準的方式, 不過要注意的是, 因為我們指定要 1 萬個字, 所以每個字是用 1 萬維的向量表示! 這一來很浪費記憶空間, 二來字和字間基本上是沒有關係的。我們可以用某種「合理」的方式, 把字壓到比較小的維度, 這些向量又代表某些意思 (比如說兩個字代表的向量角度小表相關程度大) 等等。

這聽來很複雜的事叫 "word embedding", 而事實上 Keras 會幫我們做。我們只需告訴 Keras 原來最大的數字是多少 (10000), 還有我們打算壓到幾維 (128)。

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

In [0]:
model = Sequential()

In [0]:
model.add(Embedding(10000, 128))

LSTM 層, 我們做 150 個 LSTM Cells。

In [0]:
model.add(LSTM(150))

單純透過 sigmoid 輸出。

In [0]:
model.add(Dense(1, activation='sigmoid'))

### 3.3 組裝

這次我們用 binary_crossentropy 做我們的 loss function, 另外用一個很潮的 Adam 學習法。

In [0]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

## 4. 訓練

我們用的 embedding 中, 會被 batch_size 影響輸入。輸入的 shape 會是

    (batch_size, 每筆上限)
    
也就是 (32,100) 輸出是 (32,100,128), 其中 128 是我們決定要壓成幾維的向量。

In [0]:
model.fit(x_train, y_train, 
          batch_size=32, 
          epochs=15)

Epoch 1/15
25000/25000 [==============================] - 399s 16ms/step - loss: 0.4247 - acc: 0.8024
Epoch 2/15
25000/25000 [==============================] - 389s 16ms/step - loss: 0.2653 - acc: 0.8947
Epoch 3/15
25000/25000 [==============================] - 403s 16ms/step - loss: 0.1929 - acc: 0.9270
Epoch 4/15
25000/25000 [==============================] - 422s 17ms/step - loss: 0.1390 - acc: 0.9497
Epoch 5/15
25000/25000 [==============================] - 396s 16ms/step - loss: 0.1005 - acc: 0.9638
Epoch 6/15
25000/25000 [==============================] - 390s 16ms/step - loss: 0.0679 - acc: 0.9776
Epoch 7/15
25000/25000 [==============================] - 391s 16ms/step - loss: 0.0529 - acc: 0.9814
Epoch 8/15
25000/25000 [==============================] - 397s 16ms/step - loss: 0.0386 - acc: 0.9881
Epoch 9/15
25000/25000 [==============================] - 393s 16ms/step - loss: 0.0357 - acc: 0.9886
Epoch 10/15
25000/25000 [==============================] - 390s 16ms/step - loss: 

## 5. 檢視結果

### 5.1 分數

我們照例來看看測試資料的分數。

In [0]:
score = model.evaluate(x_test, y_test)

25000/25000 [==============================] - 122s 5ms/step


In [0]:
print('測試資料的 loss:', score[0])
print('測試資料正確率:', score[1])

測試資料的 loss: 0.915229337259
測試資料正確率: 0.82688


### 5.2 儲存結果

這裡有 8 成我們可以正確分辨, 看來還不差, 照例我們把結果存檔。

In [0]:
model_json = model.to_json()
open('imdb_model_architecture.json', 
     'w').write(model_json)
model.save_weights('imdb_model_weights.h5')